# Charlotte
## to do
 - [ ] rag store moved to db
 - [ ] elasticsearch over rag?
 - [ ] long term memory anyhow
 - [ ] graph db of some kind
 - [ ] graph queries
 - [ ] rolodex type tool for storing people info
 - [ ] todo list / gtd tool
 - [ ] tool library moved to separate files



In [3]:
!pip install -qU chromadb langchain-ollama langgraph ddgs langchain-community langchain-text-splitters
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
import ast
recurse = 100 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:0.6b")
search = DuckDuckGoSearchRun()

embeddings = OllamaEmbeddings(model='mxbai-embed-large:latest')
vectorstore = InMemoryVectorStore(embedding = embeddings)


whoaminow = """
You are a brilliant synthetic assistant, and pride yourself on accuracy.
"""

taskrules = """
A text will be provided. 
Complete the tasks provided by the getNextTask tool. 
Prefer primary evidence over secondary, and secondary over tertiary evidence.
Prefer reputable information sources (scientific or historical concensus, accredited research institutions, professionals working in a relevant field) 
Continue to get the next task for this text using the getNextTask tool until there are no more tasks.
"""

steps = [
"1. List all claims which the text makes.",
"2. If possible identify who wrote it, and who they work for or if they are self-employed. Were there potential conflicts of interest?",
"3. Determine what categories this text fits",
"4. Identify the key players in the text.",
"5. Determine what this text is about.",
"6. List all claims which the text makes. Try to determine if they are intended to be factual claims.",
"7. Determine when the text was written, and when the events in the article if any take place.",
"8. Determine what evidence the author cites to support any claims.",
"9. Try to determine why the piece was written. Is it intended to inform, entertain, deceive, persuade, is there a key point it is trying to make?",
"10. If there is a key point the article was trying to make, try and figure out why this author in particular wants to accomplish this.",
"""11. Note any contradictions either internally or with accepted facts. Verify whether the accepted facts or the claim the article makes,
is likely to be correct, given available sources.""",
].reverse()

def getNextTask():
    """Get the next task."""
    task = steps.pop()
    if len(steps) > 1:
        return task + " then use addToRag to store this as a note. Then get the next task."
    if len(steps) == 1:
        return task + "then use addToRag to store this as a note. There are no more tasks."
    if len(steps) == 0:
        return "There are no more tasks."

etiquette = str.format(f"""
User: {whoaminow}
System: {taskrules}
Assistant:
""", whoaminow, taskrules)

## <RAG>



def addToRag(content:str, metadata:dict):
    """
    Stores content to the shared vector store.
    
    args:
        -content: string
        -metadata: dictionary
         example: "{'source': 'news', 'url': 'https://jacobin.com/', 'agent': 'currentagentname'}"
    """
    try:
        document_to_store = Document(
            page_content=str(content),
            metadata=metadata,
        )

        doc_splits = text_splitter.split_documents([document_to_store])
    
        vectorstore.add_documents(documents=document_to_add, embedding = embeddings)
        return "Content added."
    except Exception as e:
        return "Exception: "+str(e)+"."

## </RAG>

## <custom tools> various custom tools ##

def retrieve(query: str):
    """Retrieve information related to a query from in memory vector store. Store is currently empty on session start."""
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

from ddgs import DDGS

def search(query="aardwolf", region="us-en", safesearch="off", timelimit=None, num_results = None, page=1, backend="auto"):
    """
    Search the internet.
    Args:
        query: text search query.
        region: us-en, uk-en, ru-ru, etc. Defaults to us-en.
        safesearch: on, moderate, off. Defaults to "moderate".
        timelimit: d, w, m, y. Defaults to None.
        num_results: number of results. Defaults to None.
        page: page of results. Defaults to 1.
        backend: A single or list of backends. Defaults to "auto".
                - "all" : all engines are used
                - "auto" : "wikipedia" + random 3 engines are used
                (available engines: bing, brave, duckduckgo, google, 
                mojeek, yandex, yahoo, wikipedia)                    
    """
    result = DDGS().text(query=query, region=region, safesearch=safesearch, timelimit=timelimit, num_results=num_results, page=page, backend=backend)
    return result

## </custom tools>
agent = create_react_agent(
    model=llm,
    tools=[search, retrieve, addToRag],
    prompt=etiquette,
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
  

article = """
The Big Ole Smelly Elephant In The Substack Room
We all know its there, but it’s awkward to talk about it
By Kelly Carmichael

Jul 14, 2025

Back in 1995, through a series of bizarre circumstances, I found myself founding the first ever hair and beauty website on the Internet.

I was working as a software engineer at the time when my hairdresser begged me to build him a website. He wanted to sell two homegrown hair videos he’d created.

Back in the mid-1990s, WordPress didn’t exist. I had to develop the entire site by myself, writing HTML code. It was agonizing, but a great learning experience.

After a few months, the hairdresser, who sold tons of his videos through the site, decided it was too much work and wanted to quit.

Excuse me?

Considering he never paid me the money he owed me, provided me with any commission, or even thanked me, I was flabbergasted.

Eventually, he signed paperwork turning the site back over to me. I promptly changed the name and design, and went on from there. The website was a huge success and is still in business today after 30 years.
The hairdresser? He went out of business.

Since my nights sweating blood and tears writing feral HTML code, I’ve lurked on new websites and social media channels as they appeared on the landscape.

Sometimes, like in the case of BlueSky and Lumo, I was invited in during their Beta versions. Mostly, I just hung out in the shadows.

Over many years, checking out new sites and channels, one thing is clear. There are always good and not-so-good parts of any web property.

Substack first rolled out its new platform in August 2012. I joined in early 2013. I’d already joined Quora in January of 2012 and was a tad slow to embrace Substack. I lurked there for a while, but it wasn’t until 2017 that I officially joined the platform.

Initially, I felt so welcome at Quora that I split my writing time between my own web properties and the Q&A site. I was also very active in the early days of Twitter (X), Instagram, and YouTube.

Although I wish I could get by without any sleep, I’ve discovered there are limited hours in a day. I have to thoughtfully schedule my writing time on different platforms.

While Substack is free to use for creating and distributing free content, the platform charges a 10% platform fee on all paid subscription revenue. That’s in addition to Stripe’s payment processing fees (2.9% + $0.30 per transaction). The 10% fee applies when writers enable paid subscriptions.

If you only have a few subscribers, your platform fees are reasonable. But if you have thousands of subscribers, like some Substackers do, the fees can become substantial.

In 2023 and 2024, as many of my friends and writing buddies transitioned from other platforms to Substack, I decided to start writing there after a period of dormancy.

There are many things to love about Substack, but there is one elephant in the room that makes it awkward and uncomfortable. I’ve found that as I’ve built a strong community, many of my Substack friends openly ask me to buy a paid membership for their newsletters.

Don’t get me wrong, I’m a writer and believe that writers should get paid fairly for their work. But I follow and am connected to over 500 Substackers. I currently have paid subscriptions to ten of my closest writer friends at an average of $8/ 8/month.

Do the math. That’s a lot of money to pay. Especially since most don’t reciprocate by buying a subscription to my newsletter.

I’m constantly receiving private messages on Substack asking me to buy a subscription to different newsletters. I also receive constant requests to send tips, KoFis, or buy an array of products, from artwork and logos to private consultations.

If I bought a subscription to all 500 of the Substackers I follow at an average price of $5–8 per month, I would have to get two more full-time jobs to pay the fees.

While I love the Substack platform and the fantastic community, I constantly struggle with the fact that each and every Substack newsletter is a separate entity. Nothing is bundled together.

Each newsletter has its own independent subscription fees that are set by the individual writers. If I want to cancel my subscriptions, I have to do each one individually.

The subscriptions all produce a constant stream of content. Sometimes my email gets horribly clogged with new content. As much as I try, I can’t keep up.

Since I know all ten of the writers I subscribe to, I feel intense guilt if I don’t read, heart, comment, and restack their new work. I also have a panic attack if I consider cancelling any of them.

Time management is an ongoing struggle. How do I possibly read constant new content from ten writers? I feel I must stay on top of their work, not just to show support, but also because I’m paying for it. I hate myself when I subscribe to a publication and never read the content.

Honestly, sometimes I have subscription fatigue. Before I can snap out of it, someone else will send me a message asking me to subscribe, tip them on KoFi, or do both. I have to decline, and I feel bad. It’s frustrating, maddening, and guilt-producing.

While Substack offers a fantastic platform for creators to directly monetize their writing and connect with readers, the individual nature of each newsletter’s subscription model creates challenges for readers like me.

I love to support writers, but trying to pick who to subscribe to is stressful. While I can be polite and say no, I hate to ever let anyone down. Especially fellow writers.

As much as I love the great features Substack offers and the many fantastic writers, I find myself struggling with a love/hate relationship with the platform.

I have paid subscriptions to The Atlantic and Vanity Fair, as well as a few online news publications. I don’t mind paying for those subscriptions, just as I didn’t mind paying for this platform or Quora’s monthly fee.

That’s because I have access to the entire publication, not just one writer.

I don’t mind the fees Substack charges, but I do mind being constantly asked to buy tips or subscriptions. It makes me uncomfortable, and I feel awkward.

Yes, I understand that Substack writers are trying to make a living, but I can’t provide a paid subscription to every writer I follow.

And of course, there’s the other elephant in the room. How do you decide which writers to subscribe to versus others? I wish I had a good answer for this ongoing dilemma, but I really don’t.

For now, I just do the best I can with the budget I have and hope everyone understands the situation.
"""
buffer = []
messages.append(SystemMessage(content=article, additional_kwargs={}, response_metadata={}))
for chunk in agent.stream(
    {"messages": messages}
):
    print(chunk.content, end="|", flush=True)



AttributeError: 'dict' object has no attribute 'content'